In [32]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
from sklearn.utils import shuffle
from sklearn import svm
from sklearn.model_selection import GridSearchCV


In [33]:
#READING THE CSV FILE
dg=pd.read_csv('train.tsv',delimiter='	')
#SHUFFLING THE DATA
dg = dg.sample(frac=1).reset_index(drop=True)
df=dg.copy()
df



,2635.json,false,Says the Annies List political group supports third-trimester abortions on demand.,abortion,dwayne-bohac,State representative,Texas,republican,0,1,0.1,0.2,0.3,a mailer
0,8775.json,mostly-true,"On foreign policy, Joe Biden was against aid f...","bush-administration,foreign-policy,history,hom...",robert-gates,Secretary of Defense,NaN,none,0.0,0.0,1.0,2.0,0.0,an interview with NPR
1,3889.json,false,China owns about 29 percent of (the U.S.) debt.,china,allen-west,"U.S. representative, Florida District 22",Florida,republican,6.0,9.0,3.0,1.0,4.0,an interview
2,10303.json,true,The United States right now incarcerates more ...,criminal-justice,nicholas-kristof,NaN,NaN,none,0.0,1.0,0.0,2.0,0.0,"a broadcast of the ""Daily Show"""
3,7199.json,true,Individuals with mental illnesses die an avera...,"health-care,medicaid",terry-russell,Executive director of the National Alliance on...,Ohio,none,0.0,0.0,0.0,0.0,0.0,in an interview
4,7056.json,half-true,On support for U.S. Rep. Paul Ryans Medicare r...,"federal-budget,health-care,medicare",tommy-thompson,NaN,Wisconsin,republican,6.0,8.0,2.0,5.0,3.0,debate comments
5,11466.json,barely-true,"Two years ago motorists in Ferguson, a city of...","city-budget,diversity,legal-issues",emanuel-cleaver,U.S. House of Representatives,Missouri,democrat,1.0,0.0,0.0,0.0,0.0,on 'Up to Date'
6,3759.json,mostly-true,The Ohio Democratic Party has raised $1.1 mill...,"campaign-finance,labor",chris-redfern,Chairman of the Ohio Democratic Party,Ohio,democrat,0.0,0.0,2.0,4.0,0.0,a radio interview
7,11268.json,false,Under the Nuclear Nonproliferation Treaty enri...,"foreign-policy,nuclear",dick-cheney,NaN,NaN,republican,5.0,4.0,2.0,2.0,1.0,"an interview on MSNBC's ""Morning Joe"""
8,251.json,false,I just talked about guns. I told you what my p...,guns,mitt-romney,Former governor,Massachusetts,republican,34.0,32.0,58.0,33.0,19.0,Meet the Press interview
9,3402.json,true,Says four-year terms for City Council members ...,city-government,lee-leffingwell,Austin mayor,Texas,democrat,1.0,0.0,4.0,1.0,0.0,State of the City address.


In [34]:
df.columns=["ID","label","statement","subject","speaker","speakers_job_title","state_info","job_affiliation","barely_true","false","half_true","mostly_true","pants_on_fire","context"]

In [35]:
df


,ID,label,statement,subject,speaker,speakers_job_title,state_info,job_affiliation,barely_true,false,half_true,mostly_true,pants_on_fire,context
0,8775.json,mostly-true,"On foreign policy, Joe Biden was against aid f...","bush-administration,foreign-policy,history,hom...",robert-gates,Secretary of Defense,NaN,none,0.0,0.0,1.0,2.0,0.0,an interview with NPR
1,3889.json,false,China owns about 29 percent of (the U.S.) debt.,china,allen-west,"U.S. representative, Florida District 22",Florida,republican,6.0,9.0,3.0,1.0,4.0,an interview
2,10303.json,true,The United States right now incarcerates more ...,criminal-justice,nicholas-kristof,NaN,NaN,none,0.0,1.0,0.0,2.0,0.0,"a broadcast of the ""Daily Show"""
3,7199.json,true,Individuals with mental illnesses die an avera...,"health-care,medicaid",terry-russell,Executive director of the National Alliance on...,Ohio,none,0.0,0.0,0.0,0.0,0.0,in an interview
4,7056.json,half-true,On support for U.S. Rep. Paul Ryans Medicare r...,"federal-budget,health-care,medicare",tommy-thompson,NaN,Wisconsin,republican,6.0,8.0,2.0,5.0,3.0,debate comments
5,11466.json,barely-true,"Two years ago motorists in Ferguson, a city of...","city-budget,diversity,legal-issues",emanuel-cleaver,U.S. House of Representatives,Missouri,democrat,1.0,0.0,0.0,0.0,0.0,on 'Up to Date'
6,3759.json,mostly-true,The Ohio Democratic Party has raised $1.1 mill...,"campaign-finance,labor",chris-redfern,Chairman of the Ohio Democratic Party,Ohio,democrat,0.0,0.0,2.0,4.0,0.0,a radio interview
7,11268.json,false,Under the Nuclear Nonproliferation Treaty enri...,"foreign-policy,nuclear",dick-cheney,NaN,NaN,republican,5.0,4.0,2.0,2.0,1.0,"an interview on MSNBC's ""Morning Joe"""
8,251.json,false,I just talked about guns. I told you what my p...,guns,mitt-romney,Former governor,Massachusetts,republican,34.0,32.0,58.0,33.0,19.0,Meet the Press interview
9,3402.json,true,Says four-year terms for City Council members ...,city-government,lee-leffingwell,Austin mayor,Texas,democrat,1.0,0.0,4.0,1.0,0.0,State of the City address.


In [36]:
unique=set(df["label"])
print(unique)

{'half-true', 'false', 'true', 'pants-fire', 'mostly-true', 'barely-true'}


In [37]:
category3=df[df.label=="half-true"]
print(category3.shape)


del df["ID"]


(2114, 14)


In [38]:
df.speakers_job_title.replace(np.NaN, 'unknown', inplace=True)
df.state_info.replace(np.NaN, 'unknown', inplace=True)

In [39]:
df.dropna(inplace=True)
df.isnull().sum()

label                 0
statement             0
subject               0
speaker               0
speakers_job_title    0
state_info            0
job_affiliation       0
barely_true           0
false                 0
half_true             0
mostly_true           0
pants_on_fire         0
context               0
dtype: int64

In [40]:
def replace (s):
    if s=="mostly-true":
        return 0
    elif s=="barely-true":
        return 1
    elif s=="half-true":
        return 2
    elif s=="true":
        return 3
    elif s=="false":
        return 4
    elif s=="pants-fire":
        return 5
   
    
df["output"]=df.label.apply(replace)
output=df["output"]
unique=set(df["output"])
print(unique)

{0, 1, 2, 3, 4, 5}


In [42]:
#unique2=set(df["speaker"])
#len(unique2)
#SPEAKER
speaker=df["speaker"].values
unique2=set(speaker)
unique2=list(unique2)
dictionary2={}
count=0
for i in range(len(unique2)):

    dictionary2[unique2[i]]=count
    count+=1
    
speaker_replace=[]
for j in range (len(speaker)):
    speaker_replace.append(dictionary2[speaker[j]])
speaker_replace=np.array(speaker_replace)
speaker_replace=speaker_replace.reshape(10137,1)



#SPEAKER JOB TITLE
speaker_job=df["speakers_job_title"].values
unique3=set(speaker_job)
unique3=list(unique3)
dictionary3={}
count=0
for i in range(len(unique3)):

    dictionary3[unique3[i]]=count
    count+=1
    
speaker_job_replace=[]
for j in range (len(speaker_job)):
    speaker_job_replace.append(dictionary3[speaker_job[j]])
speaker_job_replace=np.array(speaker_job_replace)
speaker_job_replace=speaker_job_replace.reshape(10137,1)



#STATE INFO

state_info=df["state_info"].values
unique4=set(state_info)
unique4=list(unique4)
dictionary4={}
count=0
for i in range(len(unique4)):

    dictionary4[unique4[i]]=count
    count+=1
    
state_info_replace=[]
for j in range (len(state_info)):
    state_info_replace.append(dictionary4[state_info[j]])
state_info_replace=np.array(state_info_replace)
state_info_replace=state_info_replace.reshape(10137,1)


#job_affiliation

job_affiliation=df["job_affiliation"].values
unique5=set(job_affiliation)
unique5=list(unique5)
dictionary5={}
count=0
for i in range(len(unique5)):

    dictionary5[unique5[i]]=count
    count+=1
    
job_affiliation_replace=[]
for j in range (len(job_affiliation)):
    job_affiliation_replace.append(dictionary5[job_affiliation[j]])
job_affiliation_replace=np.array(job_affiliation_replace)
job_affiliation_replace=job_affiliation_replace.reshape(10137,1)








In [43]:
#CONTEXT

context=df["context"].values
unique6=set(context)
unique6=list(unique6)
dictionary6={}
count=0
for i in range(len(unique6)):

    dictionary6[unique6[i]]=count
    count+=1
    
context_replace=[]
for j in range (len(context)):
    context_replace.append(dictionary6[context[j]])
context_replace=np.array(context_replace)
context_replace=context_replace.reshape(10137,1)
context_replace




array([[ 263],
       [3795],
       [ 159],
       ...,
       [3488],
       [ 482],
       [ 296]])

In [44]:
d=df["statement"]
statement=d.values
statement.shape
subject=df["subject"].values




CREATING A TOKENIZED FORM OF DATA TO WORK UPON

In [45]:
word_data=[]

for i in range(len(statement)):
    word_data.append(statement[i])
print(len(word_data))

10137


In [46]:
word_data1=[]

for i in range(len(subject)):
    word_data1.append(subject[i])
print(len(word_data1))

10137


In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer

# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
vectorizer.fit(statement)
# summarize
print(vectorizer.vocabulary_)
print(vectorizer.idf_)
# encode document
vector = vectorizer.transform(statement)
# summarize encoded vector
print(vector.shape)
x=vector.toarray()


{'on': 7694, 'foreign': 4636, 'policy': 8335, 'joe': 6102, 'biden': 1607, 'was': 11759, 'against': 833, 'aid': 861, 'for': 4622, 'vietnam': 11629, 'said': 9559, 'the': 10942, 'fall': 4328, 'of': 7638, 'shah': 9893, 'step': 10439, 'forward': 4666, 'human': 5518, 'rights': 9401, 'in': 5659, 'iran': 5970, 'opposed': 7729, 'reagans': 8950, 'defense': 3213, 'build': 1904, 'up': 11489, 'and': 1016, 'voted': 11689, 'first': 4524, 'gulf': 5127, 'war': 11736, 'china': 2317, 'owns': 7894, 'about': 615, '29': 293, 'percent': 8123, 'debt': 3141, 'united': 11433, 'states': 10399, 'right': 9399, 'now': 7553, 'incarcerates': 5669, 'more': 7240, 'african': 824, 'americans': 985, 'as': 1207, 'percentage': 8124, 'than': 10927, 'apartheid': 1082, 'south': 10220, 'africa': 823, 'did': 3416, 'individuals': 5740, 'with': 11967, 'mental': 7024, 'illnesses': 5595, 'die': 3419, 'an': 1008, 'average': 1333, '25': 273, 'years': 12078, 'earlier': 3786, 'those': 11000, 'without': 11977, 'illness': 5594, 'support':

(10137, 12136)


In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer

# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
vectorizer.fit(subject)
# summarize
print(vectorizer.vocabulary_)
print(vectorizer.idf_)
# encode document
vector = vectorizer.transform(subject)
# summarize encoded vector
print(vector.shape)
x1=vector.toarray()



{'bush': 23, 'administration': 6, 'foreign': 74, 'policy': 122, 'history': 82, 'homeland': 83, 'security': 142, 'iraq': 91, 'military': 110, 'china': 33, 'criminal': 46, 'justice': 97, 'health': 81, 'care': 28, 'medicaid': 107, 'federal': 67, 'budget': 22, 'medicare': 108, 'city': 34, 'diversity': 54, 'legal': 100, 'issues': 94, 'campaign': 25, 'finance': 68, 'labor': 99, 'nuclear': 115, 'guns': 79, 'government': 78, 'taxes': 157, 'immigration': 88, 'candidates': 26, 'biography': 19, 'climate': 36, 'change': 31, 'energy': 61, 'economy': 57, 'education': 58, 'jobs': 96, 'small': 145, 'business': 24, 'stimulus': 153, 'social': 146, 'agriculture': 10, 'environment': 62, 'water': 171, 'crime': 45, 'infrastructure': 90, 'message': 109, 'machine': 103, '2012': 1, 'public': 129, 'voting': 169, 'record': 131, 'women': 176, 'workers': 177, 'safety': 140, 'market': 105, 'regulation': 134, 'terrorism': 159, 'state': 151, 'finances': 69, 'debt': 50, 'deficit': 51, 'elections': 60, 'corrections': 4

In [49]:
x_x=np.hstack((x,x1))
print(x_x.shape)

(10137, 12314)


In [173]:
x2=x_x[:,0:1]


numpy.ndarray

In [120]:
x2=np.hstack((x2,state_info_replace))
x2=np.hstack((x2,speaker_replace))
x2=np.hstack((x2,speaker_job_replace))
x2=np.hstack((x2,job_affiliation_replace))



In [121]:
x2=np.hstack((x2,context_replace))



In [193]:
barely_true=df["barely_true"].values
barely_true=np.array(barely_true)
barely_true=barely_true.reshape(10137,1)
x_x=np.hstack((x_x,barely_true))

half_true=df["half_true"].values
half_true=np.array(half_true)
half_true=half_true.reshape(10137,1)
x_x=np.hstack((x_x,half_true))

mostly_true=df["mostly_true"].values
mostly_true=np.array(mostly_true)
mostly_true=mostly_true.reshape(10137,1)
x_x=np.hstack((x_x,mostly_true))

pants_on_fire=df["pants_on_fire"].values
pants_on_fire=np.array(pants_on_fire)
pants_on_fire=pants_on_fire.reshape(10137,1)
x_x=np.hstack((x_x,pants_on_fire))

false=df["false"].values
false=np.array(false)
false=false.reshape(10137,1)
x_x=np.hstack((x_x,false))



In [51]:
x_x.shape

(10137, 12314)

In [52]:
y=df["output"].values

SPLITTING OUR DATA INTO TRAIN AND TEST

In [64]:
from sklearn import model_selection
x_train, x_test, y_train, y_test = model_selection.train_test_split(x_x, y, test_size = 0.35, random_state = 0)

IMPLEMENTING NAIVE BAYES ALGORITHM

In [65]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

train_score = clf.score(x_train, y_train)
test_score = clf.score(x_test, y_test)

print(test_score)

0.23111612175873733


In [69]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=12)

y_pred = rf_classifier.fit(x_train,y_train).predict(x_test)

rf_classifier.score(x_test, y_test)

0.2229425028184893